# Initialization

In [1]:
import logging, os, sys, time

import numpy as np
import matplotlib.pyplot as plt

sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
import backprojection as tdbp
from posarmctools import getNotebookLogger

sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
import posarutils.process.disp_PoSAR_img as disp
from posarutils.process.disp_PoSAR_img import OPTt
from posarutils.process.filtering import box_filter

sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
import toolsdiadem.tools as dia

# this is for the sceneReferencePoint.py module
sys.path.insert(0, "/home/pleroy/DATA/PoSAR-X/PIMA-1")

In [2]:
%matplotlib qt5
eps = np.finfo(float).eps

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
logger = getNotebookLogger(logging.INFO)

In [5]:
libraryFilename = "/home/pleroy/DEV/processing/PoSAR-MC/backprojection/ccpp/libbackprojection/liblibbackprojection.so"

In [6]:
lib = tdbp.Lib(libraryFilename)

# Load focusing configuration and data

In [64]:
#ini = "/home/pleroy/DATA/PoSAR-X/PIMA-1/2019_07_12/FOCUSING/2019_07_12_14_12_21.ini"
#ini = "/home/pleroy/DATA/PoSAR-X/PIMA-1/2019_07_12/FOCUSING/2019_07_12_14_14_43.ini"
#ini = "/home/pleroy/DATA/PoSAR-X/PIMA-1/2019_07_12/FOCUSING/2019_07_12_14_18_28.ini"
#ini = "/home/pleroy/DATA/PoSAR-X/PIMA-1/2019_07_12/FOCUSING/2019_07_12_14_12_21_center.ini"
# PIMA-2
#ini = "/home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_23/FOCUSING/2020_06_23_18_36_24.ini"
#ini = "/home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_23/FOCUSING/2020_06_23_18_38_34.ini"
#ini = "/home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_23/FOCUSING/2020_06_23_18_42_20.ini"
#ini = "/home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_23/FOCUSING/2020_06_23_18_44_48.ini"
# 2020 06 25
focusingDir = "/home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25/FOCUSING"
#ini = "2020_06_25_11_57_59.ini"
#ini = "2020_06_25_11_57_59_c2.ini"
#ini = "2020_06_25_11_57_59_c2_noWin.ini"
#ini = "2020_06_25_11_57_59_sr.ini"
ini = "2020_06_25_11_57_59_sr_noWin.ini"
#ini = "2020_06_25_12_04_34.ini"
#ini = "2020_06_25_12_11_37.ini"
focusing = tdbp.Focusing(os.path.join(focusingDir, ini), tag="Sa", alongTrack=1, pointsFile="ini")

======= Conf
rec_dir /home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25/2020_06_25_11_57_59
======= Track
======= Load analytic signal...
load /home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25/OUT/2020_06_25_11_57_59/Sa_files_0_56_rampUp_None.npy
sa.shape (57000, 5000)
existing analytic signal found and loaded
======= Load antenna positions...
load /home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25/OUT/2020_06_25_11_57_59/n_time_xyz_nav_a.npy
xa.shape (57000,)
xa_mean = 1326869.63, ya_mean = 7218883.32, za_mean = 500.65
======= Load Digital Terrain Elevation Data...
get dataset
build meshgrids
build rectBivariateSpline
======= Scene
buildSceneSlantRange
closestApproachXY 349.48924534082937, closestApproachXYZ 538.6340831872086, h0 409.85844258966085
======= Set focusingParameters
Nf 5000, Naz 57000
range from 0.00m to 1249.97m, resolution = 0.25m, oversampled = 0.025m, 


## Compute scene elevations

In [65]:
fig, ax = plt.subplots(1,1)
x = focusing.scene.X
y = focusing.scene.Y
ax.set_aspect('equal')
ax.plot(focusing.xa, focusing.ya)
ax.plot(x, y, '.k')
ax.grid()
focusing.addPtsEpsg(ax, ['c0', 'c1', 'c2', 'c3', 'c4'])
focusing.addPtsEpsg(ax, ['h0', 'h1', 'h2', 'h3'])
ax.plot(focusing.scene.centerX, focusing.scene.centerY, 'o')

# Focalization

In [66]:
t = time.time()

focusedImg  = np.zeros( (focusing.scene.nY, focusing.scene.nX), dtype=complex )
print( "focusedImg.shape = {}".format( focusedImg.shape ) )

lib.so.backProjectionOmpGroundRange_LETG(focusing.scene.X.reshape(focusing.scene.X.size), 
                                         focusing.scene.Y.reshape(focusing.scene.Y.size),
                                         focusing.sceneZ.reshape(focusing.sceneZ.size),
                                         focusing.r_over,
                                         focusing.sa.reshape(focusing.sa.size),
                                         focusing.xyz.reshape(focusing.xyz.size), 
                                         focusedImg.reshape(focusedImg.size),
                                         focusing.parameters)

elapsed = time.time() - t
print("execution time = " + str(elapsed))

focusedImg.shape = (400, 500)
execution time = 525.3961038589478


In [74]:
myImg = tdbp.Img(focusedImg)
#min_dB = -34.70, max_dB = 67.19, med_dB = 28.79

min_dB = -27.25, max_dB = 60.80, med_dB = 28.79


# Plot image

## imshow

In [68]:
cmap = 'jet'

In [80]:
left = focusing.conf.xMax
right = focusing.conf.xMin
bottom = focusing.conf.yMax
top = focusing.conf.yMin
extent = [left, right, bottom, top]
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)
#im0 = ax0.imshow(myImg.img_dB, cmap=cmap, extent=extent)
#im1 = ax1.imshow(myImg.box_dB(3), cmap=cmap, extent=extent)
im0 = ax0.imshow(myImg.img_dB, cmap=cmap, vmin=30, vmax=60)
im1 = ax1.imshow(myImg.box_dB(3), cmap=cmap)
dia.addColorBar(im0, ax0)
dia.addColorBar(im1, ax1)

In [72]:
plt.figure()
#for idx in range(345, 355, 1):
#    plt.plot(myImg.img_dB.T[idx, :], label=f"{idx}")
idx = 349
plt.plot(np.arange(focusing.conf.yMin, focusing.conf.yMax, focusing.conf.d_y), myImg.img_dB.T[idx, :], label=f"{idx}")
#idx = 211
#plt.plot(np.arange(focusing.conf.yMin, focusing.conf.yMax, focusing.conf.d_y), myImg.img_dB.T[idx, :], label=f"{idx}")
plt.grid()
plt.legend()

## pcolormesh box_filter

In [73]:
z = myImg.box_dB(1)

In [ ]:
fig, ax = plt.subplots(1,1)
x = focusing.scene.X
y = focusing.scene.Y
im = ax.pcolormesh(x, y, z, cmap='jet')
ax.set_aspect('equal')
ax.plot(focusing.xa, focusing.ya)
plt.colorbar(im)
ax.grid()
ax.plot(focusing.conf.ptsEpsg['corner'][0], focusing.conf.ptsEpsg['corner'][1], 'og', markeredgecolor = 'black')

In [ ]:
vmin = 0
vmax = 40

fig, ax = plt.subplots(1,1)
x = focusing.scene.X
y = focusing.scene.Y
im = ax.pcolormesh(x, y, z, cmap='jet', vmin=vmin, vmax=vmax)
ax.set_aspect('equal')
ax.plot(focusing.xa, focusing.ya)
plt.colorbar(im)
ax.grid()
#focusing.addPtsEpsg(ax, ['corner'])
#focusing.addPtsEpsg(ax, ['h0', 'h1', 'h2', 'h3'])

## Dynamic range

In [31]:
imgDyn = myImg.dynamicRange(dynRange=60, db=1, size=3)

min 15.3, max 58.6


In [32]:
fig, ax = plt.subplots(1,1)
im = ax.pcolormesh(x, y, z, cmap=cmap, vmin=-10, vmax=30)
dia.addColorBar(im, ax)
ax.grid()
focusing.addPtsEpsg(ax, ['corner'])
focusing.addPtsEpsg(ax, ['h0', 'h1', 'h2', 'h3'])

# Build GeoTIFF

In [33]:
tiffName = os.path.join(focusing.out_dir, focusing.conf.name + ".tif")

In [34]:
myTiff = tdbp.Tiff(tiffName, focusing, z, cmap='gray')

# Save focused image

In [81]:
myImg.save(focusing)

In [76]:
sig = 20 * np.log10(np.abs(np.fft.ifft(focusing.sa)))

In [77]:
plt.imshow(sig[500:800:, 1500:3000], cmap='jet')

In [38]:
plt.figure()
plt.plot(sig[700, :])